# HW 5.1

# Q1 (15pt) Retrieving files over HTTP/HTTPS using Response

Try this: Retrieving A file

* If you’re working with the data file above and want to break each line into separate fields, 
    * how might you do that?What other processing would you expect to do?

* In this assignment, we will try writing some code to retrieve this file and calculate the 
    * (2pt) The average rainfall over all the months and years
    * (2pt) The highest temperature in Celsius degree over all the months and years
    * (2pt) The lowest temperature in Celsius degree over all the months and years
    * (9pt) The average maximum/minimum temperature and average rainfall each year (in a table)
    
* The expected output is shown below.

* A starter code is given below, which is from the solution in Chapter 22 at 
    * https://learning.oreilly.com/library/view/the-quick-python/9781617294037/kindle_split_040.html
    
* Fill in your program below.

In [1]:
import requests
import pandas as pd
response = requests.get("http://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/heathrowdata.txt")
data = response.text
data_rows = []
rain = []
maxT = []
minT = []
tmax = {}
tmin = {}
rave = {}
years = set()


for row in data.split("\r\n")[7:]:
    fields = [x for x in row.split(" ") if x]
    year = int(fields[0])
    years.add(year)
    data_rows.append(fields)
    rain.append(float(fields[5]))
    maxT.append(float(fields[2]))
    minT.append(float(fields[3]))
    
    tmax.setdefault(year,[])
    tmin.setdefault(year,[])
    rave.setdefault(year,[])
    
    tmax[year].append(float(fields[2]))  
    tmin[year].append(float(fields[3]))  
    rave[year].append(float(fields[5]))

print("Average rainfall = {:.1f} mm".format(sum(rain)/len(rain)))
print("The highest temparature in degree = {}".format(max(maxT)))
print("The lowest temparature in degree = {}".format(min(minT)))




years = list(years)
highest_degree = {}
lowest_degree = {}
rain_year = {}


for year in years:
    highest_degree[year] = max(tmax[year])
    lowest_degree[year] = min(tmin[year])
    rain_year[year] = '%.1f'% (sum(rave[year])/len(rave[year]))
    

highest_degree = list(highest_degree.values())
lowest_degree = list(lowest_degree.values())
rain_year = list(rain_year.values())
print(pd.DataFrame({'Years':years,'Highest_Degree':highest_degree,'Lowest_Degree':lowest_degree,'Rainfall':rain_year}))


Average rainfall = 50.5 mm
The highest temparature in degree = 28.3
The lowest temparature in degree = -4.6
    Years  Highest_Degree  Lowest_Degree Rainfall
0    1948            21.7            2.2     47.0
1    1949            25.1            0.6     41.5
2    1950            23.6           -1.0     53.4
3    1951            23.0            0.8     67.3
4    1952            23.3           -0.3     51.4
..    ...             ...            ...      ...
68   2016            24.7            2.9     49.2
69   2017            24.0            0.7     47.7
70   2018            28.3            0.6     48.3
71   2019            25.5            2.0     54.4
72   2020            25.6            3.9     54.0

[73 rows x 4 columns]


# Q2: (15pt) Web Scraping with Python Using Beautiful Soup

In [10]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
weather = []
periods = []
response = requests.get('https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168')
soup = BeautifulSoup(response.text,'html.parser')
seven_day = soup.find(id = 'seven-day-forecast')
forecast_items = seven_day.find_all(class_= 'tombstone-container')
temp = seven_day.find_all(class_ = 'temp')
weathers = seven_day.find_all(class_ = 'short-desc')
for x in weathers:
    x = str(x)
    x = re.findall('>[a-zA-Z ]*<',x)
    h = ''
    for i in x:
        i = i.replace('>',' ').replace('<','')
        h = h+i
    weather.append(h)
weather_condition = []
for i in range(len(weather)):
    if weather[i] not in weather_condition:
        weather_condition.append(weather[i])
weather_condition.sort()
weather_condition.insert(0,'_'*len(weather_condition[0]))

for i in range(len(weather_condition)):
    periods.append(weather.count(weather_condition[i]))
periods[0] = '_'*len(weather_condition[0])

print(pd.DataFrame({'Weather':weather_condition,'# periods':periods}))




                                     Weather        # periods
0                            _______________  _______________
1                             Chance Showers                2
2                               Mostly Clear                1
3                              Mostly Cloudy                2
4   Mostly Cloudy then Slight Chance Showers                1
5                             Showers Likely                2
6                                      Sunny                1
